In [ ]:
!pip install kaggle

In [ ]:
import librosa
import IPython.display as player
import matplotlib.pyplot as plt
import librosa.display as disp
import os
import numpy as np

In [ ]:
from google.colab import files

# Upload the Kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c birdclef-2023

100% 4.90G/4.91G [00:52<00:00, 123MB/s]
100% 4.91G/4.91G [00:52<00:00, 99.6MB/s]


In [ ]:
!unzip -q birdclef-2023.zip -d birdclef_dataset

In [ ]:
!ls birdclef_dataset

eBird_Taxonomy_v2021.csv  sample_submission.csv  test_soundscapes  train_audio	train_metadata.csv


In [ ]:
!ls birdclef_dataset/train_audio

abethr1  beasun2  brobab1  combul2  gbesta1  hartur1  malkin1  raybar1	scrcha1  subbus1  wheslf1
abhori1  bkctch1  broman1  combuz1  gnbcam2  helgui   marsto1  rbsrob1	scthon1  supsta1  whhsaw1
abythr1  bkfruw1  brosun1  comsan   gnhsun1  hipbab1  marsun2  rebfir2	shesta1  tacsun1  whihel1
afbfly1  blacra1  brrwhe3  crefra2  gobbun1  hoopoe   mcptit1  rebhor1	sichor1  tafpri1  whrshr1
afdfly1  blacuc1  brtcha1  crheag1  gobsta5  huncis1  meypar1  reboxp1	sincis1  tamdov1  witswa1
afecuc1  blakit1  brubru1  crohor1  gobwea1  hunsun2  moccha1  reccor	slbgre1  thrnig1  wlwwar
affeag1  blaplo1  brwwar1  darbar1  golher1  joygre1  mouwag1  reccuc1	slcbou1  trobou1  wookin1
afgfly1  blbpuf2  bswdov1  darter3  grbcam1  kerspa2  ndcsun2  reedov1	sltnig1  varsun2  woosan
afghor1  blcapa2  btweye2  didcuc1  grccra1  klacuc1  nobfly1  refbar2	sobfly1  vibsta2  wtbeat1
afmdov1  blfbus1  bubwar2  dotbar1  grecor   kvbsun1  norbro1  refcro1	somgre1  vilwea1  yebapa1
afpfly1  blhgon1  butapa1  dutdov

In [ ]:
import os
import librosa
import numpy as np

# Set the path to the train_audio folder
train_audio_path = 'birdclef_dataset/train_audio'

# Function to extract features from an audio file
def extract_features(audio_path):
    try:
        # Load audio file
        aud, sr = librosa.load(audio_path)

        # Extract additional features as needed
        # For example, you might want to extract MFCCs
        rmse = librosa.feature.rms(y=aud)
    except Exception as e:
        print(f"Error processing {audio_path}: {str(e)}")
    return np.array([rmse])

# Lists to store features and labels
X_list = []
y_list = []

# Iterate through each folder in train_audio
for folder in os.listdir(train_audio_path):
    folder_path = os.path.join(train_audio_path, folder)

    # Check if the item in the folder is a directory
    if os.path.isdir(folder_path):
        # Counter to keep track of examples taken from the folder
        examples_taken = 0

        # Iterate through audio files in the folder
        for audio_file in os.listdir(folder_path):
            audio_path = os.path.join(folder_path, audio_file)

            # Check if the item in the folder is a file and has the .ogg extension
            if os.path.isfile(audio_path) and audio_file.endswith('.ogg'):
                # Extract features
                features = extract_features(audio_path)

                # Process the features as needed
                if features is not None:
                    # Save features to a dataset
                    X_list.append(features)
                    y_list.append(folder)  # Use the folder name as the label

                    # Increment the counter
                    examples_taken += 1

                    # Break the loop if two examples are taken from the folder
                    if examples_taken == 5:
                        break

# Convert lists to numpy arrays
X = np.array(X_list)
y = np.array(y_list)

# Save the dataset
np.save('X_features.npy', X)
np.save('y_labels.npy', y)


In [ ]:
X.shape,len(y)

((1271,), 1271)

In [ ]:
X = np.reshape(X, (X.shape[0], 1, 1))

In [ ]:
X.shape

(1271, 1, 1)

In [ ]:
# Print the shape of X
print("Shape of X:", X.shape)

# Print the contents of X
print("Contents of X:")
print(X)


Shape of X: (1271, 1, 1)
Contents of X:
[[[{'music_array': array([ 8.3054119e-06, -1.4026858e-05,  3.4115124e-07, ...,
          -2.9702940e-05,  1.2471619e-05,  9.5329806e-06], dtype=float32), 'sr': 22050, 'mfccs': array([[-8.2522931e+02, -8.2519794e+02, -8.2285431e+02, ...,
           -6.8971198e+02, -7.3856110e+02, -8.0420740e+02],
          [ 0.0000000e+00,  4.2622253e-02,  3.3320494e+00, ...,
            5.4491859e+01,  4.2525448e+01,  1.8678585e+01],
          [ 0.0000000e+00,  3.7494734e-02,  3.2527797e+00, ...,
            4.3190613e+01,  4.8459106e+01,  2.5032852e+01],
          ...,
          [ 0.0000000e+00, -4.2140286e-02,  1.2764510e+00, ...,
            1.4542385e+01,  1.4066024e+01,  1.0343951e+01],
          [ 0.0000000e+00, -4.4348203e-02,  9.6365952e-01, ...,
            1.9759941e+00,  3.6967330e+00,  9.0993176e+00],
          [ 0.0000000e+00, -4.3050062e-02,  6.5992492e-01, ...,
            1.1484039e+01,  1.1164937e+01,  7.6206198e+00]], dtype=float32)}            

In [ ]:
min_length = float('inf')

# Training CNN Model on RMSE Features

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd

# Set the path to the train_audio folder
train_audio_path = 'birdclef_dataset/train_audio'

# Function to extract features from an audio file
def extract_features(audio_path):
    try:
        # Load audio file
        audio, sample_rate = librosa.load(audio_path)

        # Extract RMSE features
        rmse = librosa.feature.rms(y=audio)
        # Flatten the RMSE features to a 1D array
        #rmse_flattened = rmse.flatten()
        return rmse  # Transpose to make it [samples, features]
    except Exception as e:
        print(f"Error processing {audio_path}: {str(e)}")
        return np.array([])

# Lists to store features and labels
X_list = []
y_list = []

# Specify the classes (folders) you want to include
selected_classes = ['barswa', 'comsan', 'eaywag1', 'thrnig1', 'wlwwar']

# Dictionary to keep track of examples taken from each class
examples_taken = {class_name: 0 for class_name in selected_classes}


# Convert lists to DataFrame
features = []
labels = []

for folder in selected_classes:
    folder_path = os.path.join(train_audio_path, folder)
    for audio_file in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, audio_file)
        rmse = extract_features(audio_path)
        if rmse is not None:
            features.append(rmse)
            labels.append(folder)



In [ ]:
min_length = min(array.shape[1] for array in features)

In [ ]:
X_truncated = np.array([array.flatten()[:min_length] for array in features])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


# X_padded is now ready for use in an RNN and can be converted to a NumPy array
X = X_truncated
y = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded = to_categorical(y_encoded,num_classes=5)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
num_features = X_train.shape[1]
num_classes = len(np.unique(y_encoded))

from keras.layers import BatchNormalization

model = Sequential([
    Dense(256, activation='relu', input_shape=(num_features,)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(5, activation='softmax')
])


# Choose the appropriate optimizer
optimizer = RMSprop(learning_rate=0.00001)

# Use categorical_crossentropy for multi-class classification
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Example: Decreasing batch size and adjusting epochs
history = model.fit(X_train, y_train, epochs=200, batch_size=16, validation_split=0.2, shuffle=True)


Epoch 1/200
100/100 [==============================] - 1s 5ms/step - loss: 2.5411 - accuracy: 0.1875 - val_loss: 1.7608 - val_accuracy: 0.2125
Epoch 2/200
100/100 [==============================] - 0s 3ms/step - loss: 2.4357 - accuracy: 0.2031 - val_loss: 1.9031 - val_accuracy: 0.2125
Epoch 3/200
100/100 [==============================] - 0s 3ms/step - loss: 2.4602 - accuracy: 0.2081 - val_loss: 1.9670 - val_accuracy: 0.2100
Epoch 4/200
100/100 [==============================] - 0s 3ms/step - loss: 2.4253 - accuracy: 0.2069 - val_loss: 1.9845 - val_accuracy: 0.2100
Epoch 5/200
100/100 [==============================] - 0s 3ms/step - loss: 2.4027 - accuracy: 0.2013 - val_loss: 1.9757 - val_accuracy: 0.2100
Epoch 6/200
100/100 [==============================] - 0s 3ms/step - loss: 2.4070 - accuracy: 0.2119 - val_loss: 1.9676 - val_accuracy: 0.2100
Epoch 7/200
100/100 [==============================] - 0s 3ms/step - loss: 2.4816 - accuracy: 0.1900 - val_loss: 1.9560 - val_accuracy: 0.2125

# Training CNN Model on MFCC Features

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd

# Set the path to the train_audio folder
train_audio_path = 'birdclef_dataset/train_audio'

# Function to extract features from an audio file
def extract_features(audio_path):
    try:
        y, sr = librosa.load(audio_path)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T,axis=0)
    except Exception as e:
        print("Error encountered while parsing file: ", audio_path)
        return None, None
    return np.array([mfccs_processed])

# Lists to store features and labels
X_list = []
y_list = []

# Specify the classes (folders) you want to include
selected_classes = ['barswa', 'comsan', 'eaywag1', 'thrnig1', 'wlwwar']

# Dictionary to keep track of examples taken from each class
examples_taken = {class_name: 0 for class_name in selected_classes}

# Iterate through each folder in train_audio
for folder in os.listdir(train_audio_path):
    folder_path = os.path.join(train_audio_path, folder)

    # Check if the item in the folder is a directory and is in the selected classes
    if os.path.isdir(folder_path) and folder in selected_classes:
        # Iterate through audio files in the folder
        for audio_file in os.listdir(folder_path):
            if examples_taken[folder] >= 200:
              break
            audio_path = os.path.join(folder_path, audio_file)
            print(examples_taken)
            # Check if the item in the folder is a file and has the .ogg extension
            if os.path.isfile(audio_path) and audio_file.endswith('.ogg'):
                # Extract features
                features = extract_features(audio_path)

                # Process the features as needed
                if features is not None:
                    # Check if the limit of 100 examples for the class is reached
                    if examples_taken[folder] < 100:
                        # Save features to lists
                        X_list.append(features)
                        y_list.append(folder)  # Use the folder name as the label

                        # Increment the counter for the class
                        examples_taken[folder] += 1

features = []
labels = []

for folder in selected_classes:
    folder_path = os.path.join(train_audio_path, folder)
    for audio_file in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, audio_file)
        mfccs = extract_features(audio_path)
        if mfccs is not None:
            features.append(mfccs)
            labels.append(folder)


{'barswa': 0, 'comsan': 0, 'eaywag1': 0, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 1, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 2, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 3, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 4, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 5, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 6, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 7, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 8, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 9, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 10, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 11, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 12, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 13, 'thrnig1': 0, 'wlwwar': 0}
{'barswa': 0, 'comsan': 0, 'eaywag1': 14, 't

In [ ]:
if X_train.ndim == 3:
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[2])


In [ ]:
num_features = X_train.shape[1]  # This should be the number of MFCC features, e.g., 40

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(num_features,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
50/50 [==============================] - 1s 11ms/step - loss: 36.1668 - accuracy: 0.2081 - val_loss: 6.2542 - val_accuracy: 0.3150
Epoch 2/100
50/50 [==============================] - 0s 8ms/step - loss: 25.4771 - accuracy: 0.2450 - val_loss: 4.3530 - val_accuracy: 0.4225
Epoch 3/100
50/50 [==============================] - 0s 8ms/step - loss: 16.9512 - accuracy: 0.2706 - val_loss: 3.6961 - val_accuracy: 0.4625
Epoch 4/100
50/50 [==============================] - 0s 7ms/step - loss: 10.6490 - accuracy: 0.3306 - val_loss: 2.9574 - val_accuracy: 0.4875
Epoch 5/100
50/50 [==============================] - 0s 5ms/step - loss: 7.7337 - accuracy: 0.3406 - val_loss: 2.6446 - val_accuracy: 0.5025
Epoch 6/100
50/50 [==============================] - 0s 7ms/step - loss: 5.3085 - accuracy: 0.3725 - val_loss: 2.2914 - val_accuracy: 0.5050
Epoch 7/100
50/50 [==============================] - 0s 5ms/step - loss: 3.5084 - accuracy: 0.4187 - val_loss: 1.7900 - val_accuracy: 0.5350
Epoch 8/